The code below modified from https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
Copyright belong to the great Francois Chollet. You are strongly encouraged to make a reference to this link in understanding how to set up the data directory and the working mechanism. However, due to the Keras updates, some of the codes no longer works and I manually coded all the codes and modified some places to make it run smoothly under python 3.5 and Keras 2.0. 

### Below code demonstrate how you can leverage Keras' preprocessing module to process image data augumentation

In [5]:
# image preprocessing with keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# the later 3 methods depends on PIL, which needs to be available

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

#print(x.shape)
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory

i = 0
# datagen.flow(x) x has to be in rank 4
# due to this img-array transformation
# datagen.flow_from_directory is more often used
for batch in datagen.flow(x, batch_size=1,
                                      save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

### Build a Conv2D model from scratch

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint 
from keras import backend as K

# dimensions of our images
img_width, img_height = 150, 150

# data and superparameters setup
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# a simple way to check the data format
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
# build the model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#Dropout helps reduce overfitting by preventing a layer from seeing twice the exact same pattern
model.add(Dense(1))
model.add(Activation('sigmoid'))

#model.summary()

Using TensorFlow backend.


In [2]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# setup a checkpoint to save only the best weights
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

# below is the data augmentation configuration used for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# below is the data augmentation configuration used for testing, with only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

# data generator which process the img in parallel with the training or testing
# flow_from_directory() will generate an infinite number of batches
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode='binary')

# train the model
model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size, 
    epochs=epochs, 
    callbacks=[checkpointer],
    validation_data=validation_generator, 
    validation_steps=nb_validation_samples // batch_size)


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 90s - loss: 0.6939 - acc: 0.5250 - val_loss: 0.6632 - val_acc: 0.5925
Epoch 2/50
125/125 [==============================] - 90s - loss: 0.6657 - acc: 0.5955 - val_loss: 0.6579 - val_acc: 0.5725
Epoch 3/50
125/125 [==============================] - 89s - loss: 0.6562 - acc: 0.6015 - val_loss: 0.6305 - val_acc: 0.6813
Epoch 4/50
125/125 [==============================] - 90s - loss: 0.6403 - acc: 0.6385 - val_loss: 0.6133 - val_acc: 0.6388
Epoch 5/50
125/125 [==============================] - 91s - loss: 0.6120 - acc: 0.6735 - val_loss: 0.6020 - val_acc: 0.6650
Epoch 6/50
125/125 [==============================] - 95s - loss: 0.5946 - acc: 0.6820 - val_loss: 0.5843 - val_acc: 0.6650
Epoch 7/50
125/125 [==============================] - 92s - loss: 0.5870 - acc: 0.6955 - val_loss: 0.5668 - val_acc: 0.6987
Epoch 8/50
125/125 [=============================

In [2]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Use bottleneck features from VGG16

Below demonstrate how to use bottleneck features generated from VGG16 imageNet weights. To get the bottleneck feature, you need to first pass all the training data through the VGG16 architecture, with all the weights freezed to get the feature maps before the former VGG16 FC layers, then use these bottleneck features as inputs to build a small model and train the ontop layer parameters.

Note if you are using bottleneck features, you should not use data augumentation techniques anymore and only rescaling is allowed.

In [5]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16

img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)
    
    # import the VGG16 network with FC layers removed
    # here weights are generated from imagenet dataset
    model = VGG16(include_top=False, weights='imagenet')
    
    # training set bottleneck features generating
    train_generator = datagen.flow_from_directory(
        train_data_dir, 
        target_size=(img_width, img_height), 
        batch_size=batch_size, 
        class_mode=None, 
        shuffle=False)
    
    bottleneck_features_train = model.predict_generator(
        train_generator, 
        nb_train_samples // batch_size)
    
    np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
    
    # validation set bottleneck features generating
    validation_generator = datagen.flow_from_directory(
        validation_data_dir, 
        target_size=(img_width, img_height), 
        batch_size=batch_size, 
        class_mode=None, 
        shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(
        validation_generator, 
        nb_validation_samples // batch_size)
    
    np.save(open('bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)
    
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(
        [0]*int(nb_train_samples / 2) + [1]*int(nb_train_samples / 2))
    
    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0]*int(nb_validation_samples / 2) + [1]*int(nb_validation_samples / 2))
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
save_bottleneck_features()
train_top_model()

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 8s - loss: 0.4698 - acc: 0.7965 - val_loss: 0.2886 - val_acc: 0.8762
Epoch 2/50
2000/2000 [==============================] - 7s - loss: 0.2728 - acc: 0.8835 - val_loss: 0.3415 - val_acc: 0.8175
Epoch 3/50
2000/2000 [==============================] - 7s - loss: 0.2112 - acc: 0.9190 - val_loss: 0.2428 - val_acc: 0.8988
Epoch 4/50
2000/2000 [==============================] - 7s - loss: 0.1866 - acc: 0.9280 - val_loss: 0.2307 - val_acc: 0.9137
Epoch 5/50
2000/2000 [==============================] - 7s - loss: 0.1545 - acc: 0.9390 - val_loss: 0.2468 - val_acc: 0.8988
Epoch 6/50
2000/2000 [==============================] - 8s - loss: 0.1328 - acc: 0.9495 - val_loss: 0.2555 - val_acc: 0.9075
Epoch 7/50
2000/2000 [==============================] - 9s - loss: 0.1144 - acc: 0.9520 - val_loss: 0.2780 - val_acc: 0.8975
Epoch 8/50
2000/2000 [==============================] - 8s - loss: 0.0977 - ac

### Fine tuning top layers

Below is a demonstration of fine tuning the last few layers of a trained model, some instructions from Francois are:

- in order to perform fine-tuning, all layers should start with properly trained weights: for instance you should not slap a randomly initialized fully-connected network on top of a pre-trained convolutional base. This is because the large gradient updates triggered by the randomly initialized weights would wreck the learned weights in the convolutional base. In our case this is why we first train the top-level classifier, and only then start fine-tuning convolutional weights alongside it (This means the fine tuning must come after the bottleneck implementation)
- we choose to only fine-tune the last convolutional block rather than the entire network in order to prevent overfitting, since the entire network would have a very large entropic capacity and thus a strong tendency to overfit. The features learned by low-level convolutional blocks are more general, less abstract than those found higher-up, so it is sensible to keep the first few blocks fixed (more general features) and only fine-tune the last one (more specialized features)
- fine-tuning should be done with a very slow learning rate, and typically with the SGD optimizer rather than an adaptative learning rate optimizer such as RMSProp. This is to make sure that the magnitude of the updates stays very small, so as not to wreck the previously learned features

In [3]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

# below weights actually come from the second example with bottleneck features
top_model_weights_path = 'bottleneck_fc_model.h5'


#dimensions for our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# the input_shape has to be added otherwise it will pop out an error
model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# Model build has to be done this way other than add in Keras 2.x
model = Model(inputs=model.input, outputs=top_model(model.output))

# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
# here the author use 25, which is a typo
for layer in model.layers[:15]:
    layer.trainable = False
    
#model.summary()

Model loaded


In [ ]:
# compile the model with SGD/momentum optimizer
# and a very slow learning rate
model.compile(loss='binary_crossentropy', 
                      optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
                      metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir, 
    target_size=(img_height, img_width), 
    batch_size=batch_size, 
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir, 
    target_size=(img_height, img_width), 
    batch_size=batch_size, 
    class_mode='binary')

model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples, 
    epochs=epochs, 
    validation_data=validation_generator,
    validation_steps=nb_validation_samples)

model.save_weights('fine_tuning_VGG16.h5')